Let $p_1, p_2, \ldots$ be propositional variables. 
A SAT problem, represented in [conjunctive normal form](https://en.wikipedia.org/wiki/Conjunctive_normal_form), consists in a conjunction of disjunctions of propositional variables and their complements, such as

$$
(p_1 \vee \bar{p}_2 \vee p_3) \wedge (p_2 \vee p_5) \; .
$$

We call each conjuct a [clause](https://en.wikipedia.org/wiki/Clause_(logic));
in the above example, we have two clauses, $c_1 = p_1 \vee \bar{p}_2 \vee p_3$ and 
$c_2 = p_2 \vee p_5$. 
The disjuncts in a clause are called [literals](https://en.wikipedia.org/wiki/Literal_(mathematical_logic)): 
for example, the first clause $c_1 = p_1 \vee \bar{p}_2 \vee p_3$ contains the literals $p_1$, $\bar{p}_2$, and $p_3$. 

The satisfiability question is: can we find a truth assignment to the variables that makes the expression true?
In the above case, the answer is yes: we can take: 

$$
p_1 = True, \; p_5 = True
$$

and any value for $p_2, p_3$. 

## SAT representation

To represent an instance of SAT, we represent literals, clauses, and the overall expression, as follows.

**Literals.** We represent the literal $p_k$ via the positive integer $k$, and the literal $\bar{p}_k$ via the negative integer $-k$. 

**Clauses.** We represent a clause via the set of integers representing the clause literals.  
For instance, we represent the clause $p_1 \vee \bar{p}_3 \vee p_4$ via the set $\{1, -3, 4\}$. 

**SAT problem.**  We represent a SAT problem (again, in conjunctive normal form) via the set consisting in the representation of its clauses. 
For instance, the problem 
$$
(p_1 \vee \bar{p}_2 \vee p_3) \wedge (p_2 \vee p_5)
$$
is represented by the set of sets:
$$
\{ \{1, -2, 3\}, \{2, 5\} \} \; .
$$

There are various operations that we need to do on clauses, and on the overall SAT problem, to solve it.  Thus, we encapsulate both clauses, and the SAT problem, in python classes, so we can associate the operations along with the representations.

### Clauses

We first define an auxiliary function, which tells us whether a set contains both an integer and its negative.  This will be used, for instance, to detect whether a clause contains both a literal and its complement.

In [ ]:
def has_pos_and_neg(l):
    return len(set(l)) > len({abs(x) for x in l})


This is the class representing a clause.  It is written in such a way that: 

* The list of literals is stored as a frozenset `self.literals`, so that we automatically remove duplicate literals, and so that we can easily define clause equality.
* It can be initialized either with a list of integers, or with a clause.  In the first case, the integers are the literals.  In the second case, we create a copy of the original clause. 
* A clause is true iff it contains both a literal and its complement. 
* A clause is false iff it is empty.

In [ ]:
### Defining Clauses

class Clause(object):

    def __init__(self, clause):
        """Initializes a clause.  Here, the input clause is either a list or set
        of integers, or is an instance of Clause; in the latter case, a shallow
        copy is made, so that one can modify this clause without modifying the
        original clause.
        Store the list of literals as a frozenset."""
        if isinstance(clause, Clause):
            # We use frozenset here, so that clauses are not modifiable.
            # This ensures that two equal clauses always have the same hash,
            # due also to our definition of the __hash__ method.
            self.literals = frozenset(clause.literals)
        else:
            for i in clause:
                # Sanity check.
                assert isinstance(i, int), "Not an integer: %r" % i
            self.literals = frozenset(clause)

    def __repr__(self):
        return repr(self.literals)

    def __eq__(self, other):
        return self.literals == other.literals

    def __hash__(self):
        """This will be used to be able to have sets of clauses,
        with clause equality defined on the equality of their literal sets."""
        return hash(self.literals)

    def __len__(self):
        return len(self.literals)

    @property
    def istrue(self):
        """A clause is true if it contains both a predicate and its complement."""
        return has_pos_and_neg(self.literals)

    @property
    def isfalse(self):
        """A clause is false if and only if it is empty."""
        return len(self.literals) == 0


Let us load also our testing framework.

In [ ]:
try:
    from nose.tools import assert_equal, assert_almost_equal
    from nose.tools import assert_true, assert_false
    from nose.tools import assert_not_equal
except:
    !pip install nose
    from nose.tools import assert_equal, assert_almost_equal
    from nose.tools import assert_true, assert_false
    from nose.tools import assert_not_equal


     |████████████████████████████████| 163kB 5.3MB/s 


These tests are there just to give you examples.  You have not written any code yet!  They are not tests for your work.

In [ ]:
# Tests for Clause.
c = Clause([2, -3, 4])
d = Clause([-3, 2, 4])
e = Clause(c)
assert_equal(c, d)
assert_equal(c, e)

c = Clause([2, 3, 5])
assert_false(c.istrue)
assert_false(c.isfalse)

c = Clause([-2, 5, 2])
assert_true(c.istrue)
assert_false(c.isfalse)

c = Clause([])
assert_false(c.istrue)
assert_true(c.isfalse)

### Truth assignments

We are seeking a truth assignment for the propositional variables that makes the expression true, and so, that makes each clause true.  

We represent the truth assignment that assigns True to $p_k$ via the integer $k$, and the truth assignment that assigns False to $p_k$ via $-k$.  Thus, if you have a (positive or negative) literal $i$, the truth assignment $i$ will make it true. 

We represent truth assignments to multiple variables simply as the set of assignments to individual variables. 
For example, the truth assignment that assigns True to $p_1$ and False to $p_2$ will be represented via the set $\{1, -2\}$. 


## Question 1: Define Clause simplification

To solve a SAT instance, we need to search for a truth assignment to its propositional variables that will make all the clauses true. 
We will search for such a truth assignment by trying to build it one variable at a time.  So a basic operation on a clause will be: 

> Given a clause, and a truth assignment for one variable, compute the result on the clause. 

What is the result on the clause?  Consider a clause  with representation $c$ (thus, $c$ is a set of integers) and a truth assignment $i$ (recall that $i$ can be positive or negative, depending on whether it assigns True or False to $p_i$).  There are three cases:

* If $i \in c$, then the $i$ literal of $c$ is true, and so is the whole clause. We return True to signify it. 
* If $-i \in c$, then the $-i$ literal of $c$ is false, and it cannot help make the clause true.  We return the clause $c \setminus \{-i\}$, which corresponds to the remaining ways of making the clause true under assignment $i$. 
* If neither $i$ nor $-i$ is in $c$, then we return $c$ itself, as $c$ is not affected by the truth assignment $i$. 

Based on the above discussion, implement a `simplify` method for a Clause that, given a truth assignment, returns either a simplified clause, if some literals remain, or True.

In [ ]:
# if i is in self's literals,
#   return the Boolean true
# elif -i is in self's literals,
#     change literals is equal to set of (self.literals)
#     return the Clause(change literals - {-i})
# else,
#   return self  

In [ ]:
### Exercise: define simplify

def clause_simplify(self, i):
    """Computes the result simplify the clause according to the
    truth assignment i."""
    ### YOUR CODE HERE
    if i in self.literals:
        return True
    elif -i in self.literals:
        change_literals = set(self.literals)
        return Clause(change_literals - {-i})
    else:
        return self


Clause.simplify = clause_simplify


In [ ]:
### Here you can write some testing code if you like. 

### YOUR CODE HERE

Here are some tests. 

In [ ]:
### 2 Points: simple tests for simplify.

c = Clause([1, 2, -3, 4])
# If we assign True to p_1, the whole clause is True.
assert_equal(c.simplify(1), True)

In [ ]:
### 3 points: hidden tests


In [ ]:
### 2 points: more tests for simplify

c = Clause([1, 2, -3, 4])
# If we assign False to 1 and True to 3, p_1 and p_3 are not useful
# any more to make the clause true.
assert_equal(c.simplify(-4), Clause([1, 2, -3]))

In [ ]:
### 3 points: hidden tests for simplify


In [ ]:
### 2 points: yet more tests for simplify

c = Clause([1, 2, -3, 4])
# Left unchanged.
assert_equal(c.simplify(12), c)


In [ ]:
### 3 points: hidden tests for simplify


## SAT Representation

A SAT instance consists in a set of clauses. 

The SAT instance is satisfiable if and only if there is a truth assignment to predicates that satisfies all of its clauses. 
Therefore: 

* If the SAT instance contains no clauses, it is trivially satisfiable.
* If the SAT instance contains an empty clause, it is unsatisfiable, since there is no way to satisfy that clause. 

Based on this idea, the initializer method for our SAT class will get a list of clauses as input.  It will discard the tautologically true ones (as indicated by the istrue clause method).  If there is even a single unsatisfiable clause, then we set the SAT problem to consist of only one unsatisfiable clause, as a shorthand for denoting that the SAT problem cannot be satisfied. 

We endow the SAT class with methods isfalse and istrue, that detect SAT problems that are trivially satisfiable by any truth assignment, or trivially unsatisfiable by any truth assignment. 

You will need to implement the methods _generate_candidate_assignments_, _apply_assignment_, and _solve_, which together will be used to search for a solution of the SAT instance.  These methods are discussed below. 

In [ ]:
class SAT(object):

    def __init__(self, clause_list):
        """clause_list is a list of lists (or better, an iterable of
        iterables), to represent a list or set of clauses."""
        raw_clauses = {Clause(c) for c in clause_list}
        # We do some initial sanity checking.
        # If a clause is empty, then it
        # cannot be satisfied, and the entire problem is False.
        # If a clause is true, it can be dropped.
        self.clauses = set()
        for c in raw_clauses:
            if c.isfalse:
                # Unsatisfiable.
                self.clauses = {c}
                break
            elif c.istrue:
                pass
            else:
                self.clauses.add(c)

    def __repr__(self):
        return repr(self.clauses)

    def __eq__(self, other):
        return self.clauses == other.clauses



## Question 2: Define istrue and isfalse

Define two functions (that are defined as properties), `istrue` and `isfalse`: 

* A SAT instance is true only if it contains no clauses.  This because empty clauses are not added to the SAT instance (see above initializer). 
* A SAT instance is false if it contains a false clause. 

These are simple functions; it's not a trick question. 

In [ ]:
# **istrue function**
# 1. return True if self.clause is empty otherwise False

# **isfalse function**
# 1. return False if self.istrue returns true
# 2. iterate through the clauses:
# 3.    return False if iterator is not empty/Len of the iterators is not 0
# 4. return true if for loop ends

In [ ]:
### istrue, isfalse, for SAT.

def sat_istrue(self):
    ### YOUR CODE HERE
    return True if self.clauses == set() else False

def sat_isfalse(self):
    ### YOUR CODE HERE
    if self.istrue is True:
        return False
    for i in self.clauses:
        if len(i) is not 0:
            return False
    return True

SAT.istrue = property(sat_istrue)
SAT.isfalse = property(sat_isfalse)


In [ ]:
### Here you can write some testing code if you like. 

### YOUR CODE HERE

Here are, as usual, some tests. 

In [ ]:
### 2 points: tests for is_true

s = SAT([[-1,-2,3],[2,-3],[1,-4,2,1]])
assert_false(s.istrue)
s = SAT([])
assert_true(s.istrue)

In [ ]:
### 3 points: hidden tests


In [ ]:
### 2 points: tests for is_false

s = SAT([[-1,-2,3],[2,-3],[1,-4,2,1]])
assert_false(s.isfalse)
s = SAT([])
assert_false(s.isfalse)
s = SAT([[],[2,-3],[1,-4,2,1]])
assert_true(s.isfalse)

In [ ]:
### 3 points: hidden tests


## Question 3: Generate candidate assignments

In order to solve a SAT instance, we proceed with the choice-constraint propagation-recursion setting.  Let us build the choice piece first. 
The idea is this: if we are to make true a clause $c$, we have to make true at least one of its literals.  Thus, we can pick a clause $c$, and try the truth assignment corresponding to each of its literals in turn: at least one of them should work.  Which clause is best to pick?  As in the Sudoku case, one with minimal length, so that the probability of one of its literals being true is highest. 

Based on this, write a method _generate_candidate_assignments_ in the above SAT class, which returns the list or set of literals of one of the clauses of minimal length.  These will be the truth assignments we will need to try in turn.  Below are some tests that your code should pass.

_Note:_ the solution can (but need not) be written in one line of code.

In [ ]:
# 1. create a list
# 2. iterate through the clauses (sorted by length)
# 3.     iterate through the literals of iterator1
# 4.     add the literals of iterator 1 if calling iterator1.simplify(iterator2) 
#        returns true (and break out of for loop if you want)
# 5. return all the elements from the first index of the list as a set

In [ ]:
### Definition of `generate_candidate_assignments`

def sat_generate_candidate_assignments(self):
    """Generates candidate assignments.
    The function picks one of the shortest clauses, and return as candidate assignments
    a list of its literals."""
    ### YOUR CODE HERE
    ll = []
    for i in sorted(self.clauses, key=len):
        for j in i.literals:
            if i.simplify(j) is True:
                ll.append(i.literals)
    
    return set(ll[0])


SAT.generate_candidate_assignments = sat_generate_candidate_assignments


In [ ]:
### Here you can write some testing code if you like. 

### YOUR CODE HERE

In [ ]:
### 2 points: Tests for `generate_candidate_assignments`

s = SAT([[-1,-2,3],[2,-3],[1,-4,2,1]])
assert_equal(set(s.generate_candidate_assignments()), {2, -3})



In [ ]:
### 3 points: hidden tests. 



## Question 4: Define apply_assignment

Once we pick a truth assignment from one of the literals above, we need to propagate its effect to the clauses of the SAT instance. 

Write an `apply_assignment` method in the SAT class given above, that takes as input a truth assignment $i$, and returns a new SAT object, whose clauses are obtained by simplifying the clauses of the current assignment according to $i$.  Clauses that are made true by $i$ (clauses where the _simplify_ method returns True) should not be part of the new SAT problem, since they are already satisfied. 

Note that you have _already_ written the code to apply an assignment to each clause; you just need to apply the assignment to all clauses and return the resulting SAT problem here.  Remember to return a _new_ SAT problem (use the SAT constructor to create it); do _not_ modify the current SAT problem. 

We provide below some tests for your code.

_Note:_ the solution can (but need not) be written in two lines of code.

In [ ]:
# 1. create a list
# 2. iterate through the clauses
# 3. add the simplified version of the clause{
#     if the simplified version does not return True 
#     (use the given assignment as the argument for simplify

In [ ]:
### Exercise: define `apply_assignment`

def sat_apply_assignment(self, assignment):
    """Applies the assignment to every clause, simplifying it.
    If a clause is false, the whole problem is unsatisfiable,
    and we return False.  If a clause is True, it does not need
    to be included."""
    ### YOUR CODE HERE
    ii = []
    for i in self.clauses:
        if i.simplify(assignment) is not True:
            ii.append(i.simplify(assignment))
    return SAT(ii)

SAT.apply_assignment = sat_apply_assignment


In [ ]:
### Here you can write some testing code if you like. 

### YOUR CODE HERE

In [ ]:
### 2 points: Tests for `apply_assignment`

# First, examples in which each clause is simplified and is part of the
# new SAT problem.
s = SAT([[-1, -2, 3], [2, -3], [5, -4, 2, 10]])
t = s.apply_assignment(1)
assert_equal(t, SAT([[-2, 3], [2, -3], [5, -4, 2, 10]]))

s = SAT([[2, 3], [4, 2, -3], [2]])
t = s.apply_assignment(-2)
assert_equal(t, SAT([[3], [4, -3], []]))


In [ ]:
### 3 points: hidden tests


In [ ]:
### 2 points: More tests for `apply_assignment`

# Second, an example in which some clauses are made True, and hence removed
# from the new SAT problem.
s = SAT([[-1, -2, 3], [2, -3], [5, -4, 2, 10]])
t = s.apply_assignment(-1)
assert_equal(t, SAT([[2, -3], [5, -4, 2, 10]]))

s = SAT([[2, 3, -4], [-1, -3, 5], [-3]])
t = s.apply_assignment(3)
assert_equal(t, SAT([[-1, 5], []]))

In [ ]:
### 3 points: hidden tests



## Question 5: Define solve

The main method for searching for a solution of the SAT instance is the `solve` method. 
The `solve` method takes no arguments, and should return either False, if the SAT instance is unsatisfiable, or a truth assignment that satisfies it.  The satisfying truth assignment should be returned as a set (of integers).  

The `solve` method uses _generate_candidate_assignments_ and _apply_assignment_ above. 

First, the `solve` method should check whether the SAT instance $S$ is trivially unsatisfiable (and return False) or trivially satisfiable (and return the empty set), using the `istrue` and `isfalse` methods. 
This takes care of the base cases of the search. 

If none of the above applies, `solve` must generate candiate truth assignments, and try them one by one.  
Let $a$ be the candidate assignment we are trying.  We apply $a$ to the current SAT problem via `apply_assignment`, obtaining $S'$. 
We recursively try to solve $S'$, via a call to `solve` of $S'$.  Then: 

* If the new SAT problem $S'$ has no solution, you can move on to the next candidate assignment, if any; 
* if the new SAT problem $S'$ returns as a solution a truth-assignment $L$ ($L$ is a list of integers), the solution is obtaining by combining $L$ and $a$ (returning the set obtained by adding $a$ to $L$). 


In [ ]:
# 1. return an empty set if self.istrue returns true
# 2. elif return False if calling self.isfalse returns true
# 3. iterate through clauses
# 4.     iterate through self.generate_candidate_assignments call (call this var)
# 5.           assign variable to self.apply_assignment call (with var as argument) (call it test)
# 6.           assign variable to recursive call (test.solve) (call it new)
# 7.           return each element in the new + var (as a set) if new isnt None and isnt False
# 8. return False if everything else finishes

# NOTE: for line 7, literal is an int so you need to change it into a set (if using union) otherwise ignore this note


In [ ]:
### Exercise: define `solve`

def sat_solve(self):
    """Solves a SAT instance.
    First, it checks whether the instance is false (in which case
    it returns False) or true (in which case it returns an empty
    assignment).
    If neither of these applies, generates a list of candidate
    assignments, and for each of them, applies them to the current SAT
    instance, generating a new SAT instance, and solves it.
    If the new SAT instance has a solution, merges it with the assignment,
    and returns it.  If it has no solution, tries the next candidate
    assignment.  If no candidate assignment works, returns False, as
    the SAT problem cannot be satisfied."""
    ### YOUR CODE HERE
    if self.istrue is True:
        return set()
    elif self.isfalse is True:
        return False

    for i in self.clauses:
        for g in self.generate_candidate_assignments():
            test = self.apply_assignment(g)
            new = test.solve()
            if (new is not None) and (new is not False):
                return set(new)|{g}
 
        return False

SAT.solve = sat_solve


To help you verify your code, let us write a method `apply_assignment` that, given a SAT problem, applies an assignment to it, and returns True if the SAT instance is satisfied. 

In [ ]:
def sat_verify_assignment(self, assignment):
    assert not has_pos_and_neg(assignment), "The assignment is inconsistent"
    s = self
    for i in assignment:
        s = s.apply_assignment(i)
        if s.istrue:
            return True
        if s.isfalse:
            return False
    return False

SAT.verify_assignment = sat_verify_assignment


In [ ]:
### Here you can write some testing code if you like. 

### YOUR CODE HERE

Here are some tests for `solve`. 

In [ ]:
### 2 points: A solvable problem

s = SAT([[1, 2], [-2, 2, 3], [-3, -2]])
a = s.solve()
print("Assignment:", a)
assert_true(s.verify_assignment(a))


Assignment: {1, -3}


In [ ]:
### 3 points: hidden tests



In [ ]:
### 2 points: Another solvable problem.

s = SAT([[1, 2], [-2, 3], [-3, 4], [-4, 5], [8, -1]])
a = s.solve()
print("Assignment:", a)
assert_true(s.verify_assignment(a))


Assignment: {1, 8, -4, -3, -2}


In [ ]:
### 3 points: hidden tests



In [ ]:
### 2 points: Yet another solvable problem

s = SAT([[-1, 2], [-2, 3], [-3, 1]])
a = s.solve()
print("Assignment:", a)
assert_true(s.verify_assignment(a))



Assignment: {1, 2, 3}


In [ ]:
### 3 points: hidden tests



In [ ]:
### 2 points: An unsolvable problem

s = SAT([[1], [-1, 2], [-2]])
assert_false(s.solve())


In [ ]:
### 3 points: hidden tests



In [ ]:
### 2 points: Another unsolvable problem

s = SAT([[-1, 2], [-2, 3], [-3, -1], [1]])
assert_false(s.solve())


In [ ]:
### 3 points: hidden tests



In [ ]:
### 2 points: Yet another unsolvable problem

s = SAT([[-1, 2], [-2, 3], [-3, -1], [1], [-4, -3, -2]])
assert_false(s.solve())


In [ ]:
### 3 points: hidden tests

